In [157]:
import boto3
import tempfile
import logging
import os

In [57]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [58]:
logging.debug('hello')

In [37]:
# Let's use Amazon S3
s3 = boto3.resource('s3')

In [38]:
# Tuplex needs a bucket.
# => create one tuplex-<iam-user> per default.
# => this is where stuff gets stored.

# layout bucket like this:
# tuplex-<iam-user>/notebooks
# tuplex-<iam-user>/data
# tuplex-<iam-user>/scratch

# upload lambda function as
# tuplex-runner

# -> add versioning to tuplex-runner! => allow for auto upload?


# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

aws-deepracer-3f4fbafa-e09c-412c-8491-baeb4b0bffb7
bbsn00
bmwcpo
tuplex
tuplex-public
tuplex-test


In [131]:
def current_iam_user():
    iam = boto3.resource('iam')
    user = iam.CurrentUser()
    return user.user_name.lower()

def default_lambda_name():
    return 'tuplex-lambda-runner'

def default_lambda_role():
    return 'tuplex-lambda-role'

def default_bucket_name():
    return 'tuplex-' + current_iam_user()

def current_region():
    session = boto3.session.Session()
    region = session.region_name
    return region

def setup_aws(iam_user=current_iam_user(),
              lambda_name=default_lambda_name(),
              lambda_role=default_lambda_role(),
              region=current_region()
              ):
    logging.info('Setting up AWS Lambda backend for IAM user {}'.format(iam_user))
    logging.info('Configuring backend in zone: {}'.format(region))
    
    # check if iam user is found?
    # --> skip for now, later properly authenticate using assume_role as described in
    # https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_use_switch-role-api.html
    
    # step 1: create Lambda role if not exists
    iam = boto3.resource('iam')
    
    
   
    
setup_aws()

INFO:root:Setting up AWS Lambda backend for IAM user leonhard
INFO:root:Configuring backend in zone: us-east-1


In [50]:
iam = boto3.resource('iam')
iam_client = boto3.client('iam')

In [113]:
lambda_role=default_lambda_role()

region = current_region()
overwrite = True


def create_lambda_role(iam_client, lambda_role):
    
    # Roles required for AWS Lambdas
    trust_policy = '{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Principal":{"Service":"lambda.amazonaws.com"},"Action":"sts:AssumeRole"}]}'
    lambda_access_to_s3 = '{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Action":["s3:*MultipartUpload*","s3:Get*","s3:ListBucket","s3:Put*"],"Resource":"*"}]}'
    lambda_invoke_others = '{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Action":["lambda:InvokeFunction","lambda:InvokeAsync"],"Resource":"*"}]}'

    iam_client.create_role(RoleName=lambda_role,
                           AssumeRolePolicyDocument=trust_policy,
                           Description='Auto-created Role for Tuplex AWS Lambda runner')
    iam_client.attach_role_policy(RoleName=lambda_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')
    iam_client.put_role_policy(RoleName=lambda_role, PolicyName='InvokeOtherlambdas', PolicyDocument=lambda_invoke_others)
    iam_client.put_role_policy(RoleName=lambda_role, PolicyName='LambdaAccessForS3', PolicyDocument=lambda_access_to_s3)
    logging.info('Created Tuplex AWS Lambda runner role ({})'.format(lambda_role))
    
    # check it exists
    try:
        response = iam_client.get_role(RoleName=lambda_role)
        print(response)
    except:
        raise Exception('Failed to create AWS Lambda Role')
    
def remove_lambda_role(iam_client, lambda_role):
    
    # detach policies...
    try:
        iam_client.detach_role_policy(RoleName=lambda_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')
    except Exception as e:
        logging.error('Error while detaching policy AWSLambdaBasicExecutionRole, Tuplex setup corrupted? Details: {}'.format(e))
        
    policy_names = iam_client.list_role_policies(RoleName=lambda_role)['PolicyNames']
    
    for name in policy_names:
        try:
            iam_client.delete_role_policy(RoleName=lambda_role, PolicyName=name)
        except Exception as e:
            logging.error('Error while detaching policy {}, Tuplex setup corrupted? Details: {}'.format(name, e))
        
    # delete role...
    iam_client.delete_role(RoleName=lambda_role)

def setup_lambda_role(iam_client, lambda_role, region, overwrite):
    try:
        response = iam_client.get_role(RoleName=lambda_role)
        logging.info('Found Lambda role from {}'.format(response['Role']['CreateDate']))

         # throw dummy exception to force overwrite
        if overwrite:
            remove_lambda_role(iam_client, lambda_role)
            logging.info('Overwriting existing role {}'.format(lambda_role))
            create_lambda_role(iam_client, lambda_role)

    except iam_client.exceptions.NoSuchEntityException as e:
        logging.info('Role {} was not found in {}, creating ...'.format(lambda_role, region))
        create_lambda_role(iam_client, lambda_role)

## Creating/specifying s3 scratch space

In [136]:
s3_client = boto3.client('s3', region_name=current_region())

In [128]:

bucket_names

['aws-deepracer-3f4fbafa-e09c-412c-8491-baeb4b0bffb7',
 'bbsn00',
 'bmwcpo',
 'tuplex',
 'tuplex-public',
 'tuplex-test']

In [129]:
# create bucket if it not exists (private one)

In [133]:
default_bucket_name()

'tuplex-leonhard'

In [144]:
def ensure_s3_bucket(s3_client, bucket_name, region):
    bucket_names = list(map(lambda b: b['Name'], s3_client.list_buckets()['Buckets']))
    
    if bucket_name not in bucket_names:
        logging.info('Bucket {} not found, creating (private bucket) in {} ...'.format(bucket_name, region))
        
        # bug in boto3: 
        if region == current_region():
            s3_client.create_bucket(Bucket=bucket_name)
            logging.info('Bucket {} created in {}'.format(bucket_name, region))
        else:
            location = {'LocationConstraint': region.strip()}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
            logging.info('Bucket {} created in {}'.format(bucket_name, region))
    else:
        logging.info('Found bucket {}'.format(bucket_name))
    
ensure_s3_bucket(s3_client, default_bucket_name(), current_region())

INFO:root:Found bucket tuplex-leonhard


### Creating/uploading actual lambda function

In [114]:
lambda_client = boto3.client('lambda')

In [156]:
lambda_function_name=default_lambda_name()
lambda_zip_file = './tplxlam.zip'

try:
    response = lambda_client.get_function(FunctionName=lambda_function_name)
    print(response)
except lambda_client.exceptions.ResourceNotFoundException as e:
    logging.info('Function {} was not found in {}, uploading ...'.format(lambda_function_name, region))
    

INFO:root:Function tuplex-lambda-runner was not found in us-east-1, uploading ...


In [159]:
def sizeof_fmt(num, suffix="B"):
    # from https://stackoverflow.com/questions/1094841/get-human-readable-version-of-file-size
    for unit in ["", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"]:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"


def upload_lambda(lambda_function_name, lambda_zip_file, overwrite=False, s3_client=None, s3_scratch_space=None):
    # AWS only allows 50MB to be uploaded directly via request. Else, requires S3 upload.
    
    ZIP_UPLOAD_LIMIT_SIZE=50000000 
    
    if not os.path.isfile(lambda_zip_file):
        raise Exception('Could not find local lambda zip file {}'.format(lambda_zip_file))
    file_size = os.stat(lambda_zip_file).st_size
    if file_size < ZIP_UPLOAD_LIMIT_SIZE:
        logging.info('Found packaged lambda ({})'.format(sizeof_fmt(file_size)))
        
        # upload directly
        
    else:
        if s3_client is None or s3_scratch_space is None:
            raise Exception("Local packaged lambda to large to upload directly, " \
                            "need S3. Please specify S3 client + scratch space")
        # upload to s3 temporarily
        
        # delete temp s3 file after delete.

46065298

In [ ]:
# need to specify the 

In [151]:
!aws s3 cp s3://tuplex-public/tplxlam.zip . --request-payer requester

fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden


In [149]:
!aws s3 cp s3://tuplex-public/tplxlam.zip .

fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden


In [152]:
# Note: S3 will give fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden in case.

In [112]:
response = iam_client.get_role(RoleName=lambda_role)
print(response)

{'Role': {'Path': '/', 'RoleName': 'tuplex-lambda-role', 'RoleId': 'AROAYRTVOQK5OLPIZZBZC', 'Arn': 'arn:aws:iam::587583095482:role/tuplex-lambda-role', 'CreateDate': datetime.datetime(2021, 11, 3, 19, 28, 9, tzinfo=tzutc()), 'AssumeRolePolicyDocument': {'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'lambda.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}, 'Description': 'Auto-created Role for Tuplex AWS Lambda runner', 'MaxSessionDuration': 3600, 'RoleLastUsed': {}}, 'ResponseMetadata': {'RequestId': '9e88216b-d2ce-4051-9ed3-17070ca499d6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9e88216b-d2ce-4051-9ed3-17070ca499d6', 'content-type': 'text/xml', 'content-length': '905', 'date': 'Wed, 03 Nov 2021 19:29:40 GMT'}, 'RetryAttempts': 0}}


In [106]:
iam_client.list_role_policies(RoleName=lambda_role)

{'PolicyNames': ['InvokeOtherlambdas', 'LambdaAccessForS3'],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': 'ee166a86-0c58-421d-9b5d-1b2db11337f7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ee166a86-0c58-421d-9b5d-1b2db11337f7',
   'content-type': 'text/xml',
   'content-length': '424',
   'date': 'Wed, 03 Nov 2021 19:24:01 GMT'},
  'RetryAttempts': 0}}

In [ ]:
iam_client.attach_role_policy(RoleName=lambda_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

In [96]:
help(iam_client.put_role_policy)

Help on method put_role_policy in module botocore.client:

put_role_policy(*args, **kwargs) method of botocore.client.IAM instance
    Adds or updates an inline policy document that is embedded in the specified IAM role.
    
     
    
    When you embed an inline policy in a role, the inline policy is used as part of the role's access (permissions) policy. The role's trust policy is created at the same time as the role, using  CreateRole . You can update a role's trust policy using  UpdateAssumeRolePolicy . For more information about IAM roles, see `Using roles to delegate permissions and federate identities <https://docs.aws.amazon.com/IAM/latest/UserGuide/roles-toplevel.html>`__ .
    
     
    
    A role can also have a managed policy attached to it. To attach a managed policy to a role, use  AttachRolePolicy . To create a new managed policy, use  CreatePolicy . For information about policies, see `Managed policies and inline policies <https://docs.aws.amazon.com/IAM/latest/User

In [93]:
remove_lambda_role(iam_client, 'tuplex-lambda-role')

In [76]:
!cat /var/folders/l7/8zgzcszx7z5gk7kk92f6nc1c0000gn/T/tmp8qrc12_k

cat: /var/folders/l7/8zgzcszx7z5gk7kk92f6nc1c0000gn/T/tmp8qrc12_k: No such file or directory


In [65]:
help(iam_client.create_role)

Help on method create_role in module botocore.client:

create_role(*args, **kwargs) method of botocore.client.IAM instance
    Creates a new role for your AWS account. For more information about roles, see `IAM roles <https://docs.aws.amazon.com/IAM/latest/UserGuide/WorkingWithRoles.html>`__ . For information about quotas for role names and the number of roles you can create, see `IAM and STS quotas <https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_iam-quotas.html>`__ in the *IAM User Guide* .
    
    
    
    See also: `AWS API Documentation <https://docs.aws.amazon.com/goto/WebAPI/iam-2010-05-08/CreateRole>`_
    
    
    **Request Syntax** 
    ::
    
      response = client.create_role(
          Path='string',
          RoleName='string',
          AssumeRolePolicyDocument='string',
          Description='string',
          MaxSessionDuration=123,
          PermissionsBoundary='string',
          Tags=[
              {
                  'Key': 'string',
             

In [10]:
iam_client = boto3.client('iam')

In [12]:
iam = boto3.resource('iam')
account_summary = iam.AccountSummary()

In [14]:
account_summary.load()

In [17]:
account_summary.get_available_subresources()

[]

In [18]:
account_summary.summary_map

{'GroupPolicySizeQuota': 5120,
 'InstanceProfilesQuota': 1000,
 'Policies': 3,
 'GroupsPerUserQuota': 10,
 'InstanceProfiles': 1,
 'AttachedPoliciesPerUserQuota': 10,
 'Users': 3,
 'PoliciesQuota': 1500,
 'Providers': 0,
 'AccountMFAEnabled': 0,
 'AccessKeysPerUserQuota': 2,
 'AssumeRolePolicySizeQuota': 2048,
 'PolicyVersionsInUseQuota': 10000,
 'GlobalEndpointTokenVersion': 1,
 'VersionsPerPolicyQuota': 5,
 'AttachedPoliciesPerGroupQuota': 10,
 'PolicySizeQuota': 6144,
 'Groups': 2,
 'AccountSigningCertificatesPresent': 0,
 'UsersQuota': 5000,
 'ServerCertificatesQuota': 20,
 'MFADevices': 0,
 'UserPolicySizeQuota': 2048,
 'PolicyVersionsInUse': 23,
 'ServerCertificates': 0,
 'Roles': 18,
 'RolesQuota': 1000,
 'SigningCertificatesPerUserQuota': 2,
 'MFADevicesInUse': 0,
 'RolePolicySizeQuota': 10240,
 'AttachedPoliciesPerRoleQuota': 10,
 'AccountAccessKeysPresent': 1,
 'GroupsQuota': 300}

In [20]:
user = iam.CurrentUser()

In [22]:
user.user_name

'Leonhard'

In [23]:
user.user_id

'AIDAIJ6K567DOELIXHE52'

In [25]:
user.get_available_subresources()

[]